In [2]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [39]:
movies = pd.read_csv("ml-1m/movies.dat", 
                 sep="::", 
                 skiprows=0, 
                 names=['MovieID','Title','Genres'],
                 engine='python'                 
                )
users = pd.read_csv("ml-1m/users.dat", 
                 sep="::", 
                 skiprows=0, 
                 names=['UserID','Gender','Age','Occupation','ZipCode'],
                 engine='python'                 
                )
ratings = pd.read_csv("ml-1m/ratings.dat", 
                 sep="::", 
                 skiprows=0, 
                 names=['UserID','MovieID','Rating','Timestamp'],
                 engine='python'                 
                )

In [41]:
movies.head(3)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [27]:
users.head(3)

,UserID,Gender,Age,Occupation,Zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [32]:
ratings.head(3)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [42]:
movies_years = movies.Title.str.extract(r"\((\d{4})\)$", expand=True)
movies_titles = movies.Title.str.extract(r"(.*) \(\d{4}\)$", expand=True)
movies_titles.head()

,0
0,Toy Story
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II


In [43]:
movies["Year"] = movies_years.astype(int)
movies["Title"] = movies_titles

In [49]:
movies.head()

,MovieID,Title,Year,Genres
0,1,Toy Story,1995,Animation|Children's|Comedy
1,2,Jumanji,1995,Adventure|Children's|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama
4,5,Father of the Bride Part II,1995,Comedy


In [15]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 4 columns):
MovieID    3883 non-null int64
Title      3882 non-null object
Genres     3883 non-null object
Year       3883 non-null int32
dtypes: int32(1), int64(1), object(2)
memory usage: 106.3+ KB


In [45]:
movies.columns[[0,1,2,3]]

Index(['MovieID', 'Title', 'Genres', 'Year'], dtype='object')

In [48]:
movies = movies.reindex(columns = movies.columns[[0,1,3,2]])

In [50]:
movies.head()

,MovieID,Title,Year,Genres
0,1,Toy Story,1995,Animation|Children's|Comedy
1,2,Jumanji,1995,Adventure|Children's|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama
4,5,Father of the Bride Part II,1995,Comedy


In [73]:
len(movies)

3883

In [51]:
movies.groupby("Year").size()

Year
1919      3
1920      2
1921      1
1922      2
1923      3
1925      6
1926      8
1927      6
1928      3
1929      3
1930      7
1931      7
1932      7
1933      7
1934      7
1935      6
1936      8
1937     11
1938      6
1939     11
1940     19
1941     11
1942     13
1943     10
1944     13
1945     11
1946     13
1947     14
1948     12
1949     10
       ... 
1971     26
1972     22
1973     29
1974     28
1975     21
1976     21
1977     22
1978     30
1979     32
1980     41
1981     43
1982     50
1983     35
1984     60
1985     65
1986    104
1987     71
1988     69
1989     60
1990     77
1991     60
1992    102
1993    165
1994    257
1995    342
1996    345
1997    315
1998    337
1999    283
2000    156
Length: 81, dtype: int64

In [99]:
tabela = users.groupby(["Age","Gender"]).agg({"UserID": "count"}).rename(columns={"UserID": "UserCount"})
# tabela.columns = ["Age","Gender","USerCount"]
# tabela.columns = np.r_[tabela.columns[:-1], ["UserCount"]]
tabela.head()

UserCount
Age Gender           
1   F              78
    M             144
18  F             298
    M             805
25  F             558

In [98]:
# tabela.replace({78: 0, 144: 1})

In [92]:
tabela.reset_index().pivot("Age","Gender","UserCount")

Gender,F,M
Age,,
1,78,144
18,298,805
25,558,1538
35,338,855
45,189,361
50,146,350
56,102,278


In [62]:
pd.crosstab(users.Age, users.Gender)

Gender,F,M
Age,,
1,78,144
18,298,805
25,558,1538
35,338,855
45,189,361
50,146,350
56,102,278


In [72]:
movies.head()

,MovieID,Title,Genres,Years
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [101]:
movie_genres=set(
    movies.Genres.str.split("|").sum()
)
movie_genres

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [109]:
movies.Genres.str.get_dummies(sep="|")

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
7,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [115]:
movies.Genres.str.get_dummies(sep="|").sum().sort_values(ascending=False)

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Western          68
Fantasy          68
Film-Noir        44
dtype: int64

In [116]:
%timeit movies.Genres.str.get_dummies(sep="|").sum().sort_values(ascending=False)

208 ms ± 2.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [111]:
movies.Genres.head()

0     Animation|Children's|Comedy
1    Adventure|Children's|Fantasy
2                  Comedy|Romance
3                    Comedy|Drama
4                          Comedy
Name: Genres, dtype: object

In [21]:
movies.Genres.str.split("|", expand=True)

,0,1,2,3,4,5
0,Animation,Children's,Comedy,None,None,None
1,Adventure,Children's,Fantasy,None,None,None
2,Comedy,Romance,None,None,None,None
3,Comedy,Drama,None,None,None,None
4,Comedy,None,None,None,None,None
5,Action,Crime,Thriller,None,None,None
6,Comedy,Romance,None,None,None,None
7,Adventure,Children's,None,None,None,None
8,Action,None,None,None,None,None
9,Action,Adventure,Thriller,None,None,None


In [22]:
movies.Genres.str.split("|", expand=True).values.flatten()

array(['Animation', "Children's", 'Comedy', ..., None, None, None],
      dtype=object)

In [25]:
pd.Series(movies.Genres.str.split("|", expand=True).values.flatten()).value_counts()

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Fantasy          68
Western          68
Film-Noir        44
dtype: int64

In [117]:
%timeit pd.Series(movies.Genres.str.split("|", expand=True).values.flatten()).value_counts()

11.9 ms ± 340 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [118]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [27]:
movies.head()

,MovieID,Title,Genres,Year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [132]:
MovieID_Rating = ratings.groupby("MovieID").Rating.mean()
max_rating = MovieID_Rating.max()
MovieID_Rating_highest = pd.DataFrame(MovieID_Rating[MovieID_Rating == max_rating])
MovieID_Rating_highest

,Rating
MovieID,
787,5.0
989,5.0
1830,5.0
3172,5.0
3233,5.0
3280,5.0
3382,5.0
3607,5.0
3656,5.0


In [137]:
pd.merge(MovieID_Rating_highest, movies.set_index("MovieID"), on="MovieID")

,Rating,Title,Year,Genres
MovieID,,,,
787,5.0,"Gate of Heavenly Peace, The",1995,Documentary
989,5.0,Schlafes Bruder (Brother of Sleep),1995,Drama
1830,5.0,Follow the Bitch,1998,Comedy
3172,5.0,Ulysses (Ulisse),1954,Adventure
3233,5.0,Smashing Time,1967,Comedy
3280,5.0,"Baby, The",1973,Horror
3382,5.0,Song of Freedom,1936,Drama
3607,5.0,One Little Indian,1973,Comedy|Drama|Western
3656,5.0,Lured,1947,Crime


In [41]:
result = pd.merge(ratings,movies, on="MovieID")[["Title","Rating"]]
result.groupby("Title").mean().sort_values(by="Rating",ascending=False).head(14)

,Rating
Title,
Ulysses (Ulisse) (1954),5.000000
Lured (1947),5.000000
Follow the Bitch (1998),5.000000
Bittersweet Motel (2000),5.000000
Song of Freedom (1936),5.000000
One Little Indian (1973),5.000000
Smashing Time (1967),5.000000
Schlafes Bruder (Brother of Sleep) (1995),5.000000
"Gate of Heavenly Peace, The (1995)",5.000000


In [146]:
MovieID_Rates_agg = ratings.groupby("MovieID")\
       .agg({"MovieID": "count", "Rating": "mean"})\
       .rename(columns={"MovieID": "RatesCount", "Rating": "RatingMean"})
MovieID_Rates_agg.head()

,RatesCount,RatingMean
MovieID,,
1,2077,4.146846
2,701,3.201141
3,478,3.016736
4,170,2.729412
5,296,3.006757


In [174]:
MovieID_Rates_agg = MovieID_Rates_agg[MovieID_Rates_agg.RatesCount > 100]
# max_rating = MovieID_Rates_agg.RatingMean.max()
# MovieID_Rates_agg[MovieID_Rates_agg.RatingMean == max_rating]
movies.set_index("MovieID").loc[MovieID_Rates_agg.RatingMean.idxmax()]

Title     Seven Samurai (The Magnificent Seven) (Shichin...
Year                                                   1954
Genres                                         Action|Drama
Name: 2019, dtype: object

In [228]:
Gen_MID_avgR = pd.merge(users[["UserID", "Gender"]], ratings)\
                 .groupby(["Gender", "MovieID"])\
                 .agg({"Rating": "mean", "UserID": "count"})\
                 .rename(columns={"Rating": "AvgRating", 'UserID': "UserCount"})
F_movies = Gen_MID_avgR.loc["F"]
M_movies = Gen_MID_avgR.loc["M"]
M_movies[M_movies.UserCount > 100].AvgRating.idxmax()

858

In [234]:
Gen_MID_avgR[Gen_MID_avgR.UserCount > 100]\
            .sort_values(by="AvgRating", ascending = False)\
            .reset_index()\
            .drop_duplicates(subset=["Gender"])

,Gender,MovieID,AvgRating,UserCount
0,F,745,4.644444,180
2,M,858,4.583333,1740


In [225]:
F_movies

,AvgRating,UserCount
MovieID,,
1,4.187817,591
2,3.278409,176
3,3.073529,136
4,2.976471,85
5,3.212963,108
6,3.682171,129
7,3.588235,204
8,3.357143,28
9,2.100000,10


In [218]:
F_favorite_MovieID = F_movies[F_movies.UserCount > 100].UserCount.idxmax()
M_favorite_MovieID = M_movies[M_movies.UserCount > 100].UserCount.idxmax()

In [221]:
F_favorite_Movie = movies.set_index("MovieID").loc[F_favorite_MovieID]
M_favorite_Movie = movies.set_index("MovieID").loc[M_favorite_MovieID]

In [222]:
F_favorite_Movie

Title     American Beauty
Year                 1999
Genres       Comedy|Drama
Name: 2858, dtype: object

In [223]:
M_favorite_Movie

Title     American Beauty
Year                 1999
Genres       Comedy|Drama
Name: 2858, dtype: object

In [237]:
pd.merge(
    pd.merge(users[["UserID", "Age"]], ratings[["UserID", "MovieID"]]),
             movies[["MovieID", "Year"]]
).groupby("Age").Year.mean()

Age
1     1988.981699
18    1989.701982
25    1987.972972
35    1984.965478
45    1983.500520
50    1982.483211
56    1981.549097
Name: Year, dtype: float64

In [238]:
movies.columns

Index(['MovieID', 'Title', 'Year', 'Genres'], dtype='object')